In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName("PySparkML").getOrCreate()

In [3]:
spark

In [4]:
#train_data - путь к файлу с данными для обучения модели
#test_data - путь к файлу с данными для оценки качества 
train_df =  spark.read.parquet("train.parquet")
test_df =  spark.read.parquet("test.parquet")

#(training_data, test_data) = feature_vector.randomSplit([0.8, 0.2],seed = 42)

In [7]:
test_df.show(5)

+-----+---------------------+---------+------+------+----------------+---------+-----------------+
|ad_id|target_audience_count|has_video|is_cpm|is_cpc|         ad_cost|day_count|              ctr|
+-----+---------------------+---------+------+------+----------------+---------+-----------------+
|    2|     11012.2068140534|        1|     1|     0|196.691891825393|       17| 0.50005065193925|
|    3|     9923.69112524699|        1|     1|     0|202.617038691842|       15|0.637132195277704|
|    4|     10202.3140990505|        1|     1|     0|203.496891469936|       15|0.783706394973096|
|   10|     10239.9431887051|        1|     1|     0|195.804239443196|       15| 1.01044552869544|
|   13|     8373.52511906263|        1|     1|     0|202.221614839989|       13| 1.05570252090352|
+-----+---------------------+---------+------+------+----------------+---------+-----------------+
only showing top 5 rows



In [7]:
train_feature = VectorAssembler(inputCols=train_df.columns[:-1], outputCol="features")
test_feature = VectorAssembler(inputCols=test_df.columns[:-1], outputCol="features")

train_vector= train_feature.transform(train_df)
test_vector =  test_feature.transform(test_df)

train_vector.show(5)
test_vector.show(5)

+-----+---------------------+---------+------+------+----------------+---------+-----------------+--------------------+
|ad_id|target_audience_count|has_video|is_cpm|is_cpc|         ad_cost|day_count|              ctr|            features|
+-----+---------------------+---------+------+------+----------------+---------+-----------------+--------------------+
|    1|     10707.2440058622|        1|     1|     0|201.829292651124|       15|0.431740082807281|[1.0,10707.244005...|
|    5|     10643.3872649482|        1|     1|     0|192.577221699704|       15|0.809264519216201|[5.0,10643.387264...|
|    6|     11418.7085911347|        1|     1|     0|204.104562956739|       11|0.909738306804039|[6.0,11418.708591...|
|    7|     10109.3278687796|        1|     1|     0|194.255798599684|       12|0.941221039774456|[7.0,10109.327868...|
|    8|     10665.1119991977|        1|     1|     0|202.658042557742|       14|0.986790019690954|[8.0,10665.111999...|
+-----+---------------------+---------+-

## Linear regression

In [26]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='ctr', maxIter=40, regParam=0.4, elasticNetParam=0.8)

lrModel = lr.fit(train_vector)

trainingSummary = lrModel.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)


RMSE: 0.411738


In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lrModel.transform(test_vector)

# Select example rows to display.
predictions.select("prediction", "ctr", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


+------------------+-----------------+--------------------+
|        prediction|              ctr|            features|
+------------------+-----------------+--------------------+
|3.5941318105996407| 0.50005065193925|[2.0,11012.206814...|
|3.7057827093344433|0.637132195277704|[3.0,9923.6911252...|
|3.6772150445911507|0.783706394973096|[4.0,10202.314099...|
|3.6734090137674107| 1.01044552869544|[10.0,10239.94318...|
| 3.864862104983643| 1.05570252090352|[13.0,8373.525119...|
+------------------+-----------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.409881


## DecisionTreeRegressor

In [38]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator


dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)
model = dt.fit(train_vector)

predictions = model.transform(test_vector)

# Select example rows to display.
predictions.select("prediction", "ctr", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

print(model)


+------------------+-----------------+--------------------+
|        prediction|              ctr|            features|
+------------------+-----------------+--------------------+
|2.6008891061065094| 0.50005065193925|[2.0,11012.206814...|
|2.6008891061065094|0.637132195277704|[3.0,9923.6911252...|
|2.6480997981072663|0.783706394973096|[4.0,10202.314099...|
|2.6008891061065094| 1.01044552869544|[10.0,10239.94318...|
|2.6008891061065094| 1.05570252090352|[13.0,8373.525119...|
+------------------+-----------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.09064
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_d56eff9ca5c0, depth=5, numNodes=63, numFeatures=7


In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_vector)

# Make predictions.
predictions = model.transform(test_vector)

# Select example rows to display.
predictions.select("prediction", "ctr", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only

+------------------+-----------------+--------------------+
|        prediction|              ctr|            features|
+------------------+-----------------+--------------------+
|2.7689802991535255| 0.50005065193925|[2.0,11012.206814...|
|2.7460638748768043|0.637132195277704|[3.0,9923.6911252...|
|2.7460638748768043|0.783706394973096|[4.0,10202.314099...|
|2.7689802991535255| 1.01044552869544|[10.0,10239.94318...|
|2.8702843357110996| 1.05570252090352|[13.0,8373.525119...|
+------------------+-----------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.115241
RandomForestRegressionModel: uid=RandomForestRegressor_2f9bee79f9a4, numTrees=20, numFeatures=7


In [39]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

rf = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_vector)

# Make predictions.
predictions = model.transform(test_vector)

# Select example rows to display.
predictions.select("prediction", "ctr", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only

+------------------+-----------------+--------------------+
|        prediction|              ctr|            features|
+------------------+-----------------+--------------------+
|2.4524957194441006| 0.50005065193925|[2.0,11012.206814...|
|2.4524957194441006|0.637132195277704|[3.0,9923.6911252...|
|2.4524957194441006|0.783706394973096|[4.0,10202.314099...|
| 2.455958038077616| 1.01044552869544|[10.0,10239.94318...|
|2.4524957194441006| 1.05570252090352|[13.0,8373.525119...|
+------------------+-----------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.0795267
GBTRegressionModel: uid=GBTRegressor_d5987e3333eb, numTrees=20, numFeatures=7


In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression


def estimate_process(regression_model, train_vector, test_vector):
    feature = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
    
    # Chain indexer and forest in a Pipeline
    pipeline = Pipeline(stages=[regression_model])

    # Train model.  This also runs the indexer.
    model = pipeline.fit(train_vector)

    # Make predictions.
    predictions = model.transform(test_vector)

    rfModel = model.stages[0]
    print(rfModel)  # summary only 
    
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

    

lr = LinearRegression(featuresCol='features', labelCol='ctr', maxIter=40, regParam=0.4, elasticNetParam=0.8)    
gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)    

regression_models = [lr, gbt, rf, dt]
for rm in regression_models:
    estimate_process(rm, train_vector, test_vector)       

LinearRegressionModel: uid=LinearRegression_b34925cdc65d, numFeatures=7
Root Mean Squared Error (RMSE) on test data = 0.409881
GBTRegressionModel: uid=GBTRegressor_9c39608d97ac, numTrees=20, numFeatures=7
Root Mean Squared Error (RMSE) on test data = 0.0896609
RandomForestRegressionModel: uid=RandomForestRegressor_27105c2dd560, numTrees=20, numFeatures=7
Root Mean Squared Error (RMSE) on test data = 0.112729
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_d6eb9607a908, depth=5, numNodes=63, numFeatures=7
Root Mean Squared Error (RMSE) on test data = 0.0818873


In [5]:
def get_features_vector(df):
    return VectorAssembler(inputCols=df.columns[:-1], outputCol="features")

# prepare vectors
train_vector= get_features_vector(train_df).transform(train_df)
test_vector =  get_features_vector(test_df).transform(test_df)

train_vector.show(5)

+-----+---------------------+---------+------+------+----------------+---------+-----------------+--------------------+
|ad_id|target_audience_count|has_video|is_cpm|is_cpc|         ad_cost|day_count|              ctr|            features|
+-----+---------------------+---------+------+------+----------------+---------+-----------------+--------------------+
|    1|     10707.2440058622|        1|     1|     0|201.829292651124|       15|0.431740082807281|[1.0,10707.244005...|
|    5|     10643.3872649482|        1|     1|     0|192.577221699704|       15|0.809264519216201|[5.0,10643.387264...|
|    6|     11418.7085911347|        1|     1|     0|204.104562956739|       11|0.909738306804039|[6.0,11418.708591...|
|    7|     10109.3278687796|        1|     1|     0|194.255798599684|       12|0.941221039774456|[7.0,10109.327868...|
|    8|     10665.1119991977|        1|     1|     0|202.658042557742|       14|0.986790019690954|[8.0,10665.111999...|
+-----+---------------------+---------+-

In [27]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

def get_gbt_param_builder(gbt):
    return ParamGridBuilder()\
        .addGrid(gbt.maxDepth, [2, 3, 4, 5])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()
#        .addGrid(gbt.numTrees, [3, 6, 9, 12, 15, 18, 20, 22])\

def get_rf_param_builder(rf):
    return ParamGridBuilder()\
        .addGrid(rf.maxDepth, [2, 3, 4, 5])\
        .addGrid(rf.numTrees, [19, 20, 21])\
        .build()

def get_dt_param_builder(dt):
    return ParamGridBuilder()\
        .addGrid(dt.maxDepth, [4, 5, 6])\
        .addGrid(dt.minInfoGain, [0.0, 0.1, 0.2])\
        .addGrid(dt.maxBins, [25, 28, 32])\
        .build()
#        .addGrid(dt.numTrees, [3, 6, 9, 12, 15, 18, 20, 22])\


def get_best_model(estimator, paramGrid, evaluator, training_data):
    tvs = TrainValidationSplit(estimator=estimator,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

    # Run TrainValidationSplit, and choose the best set of parameters.
    model = tvs.fit(training_data)
    return model.bestModel



lr = LinearRegression(featuresCol='features', labelCol='ctr', maxIter=40, regParam=0.4, elasticNetParam=0.8)    
gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)    

regression_models = [(gbt, get_gbt_param_builder(gbt)), (rf, get_rf_param_builder(rf)), (dt, get_dt_param_builder(dt))]
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
best_models = []

for (model, params) in regression_models:
    best_models.append(get_best_model(model, params, evaluator, train_vector))


In [28]:
best_models

[GBTRegressionModel: uid=GBTRegressor_b4119c566cad, numTrees=6, numFeatures=7,
 RandomForestRegressionModel: uid=RandomForestRegressor_1ec5abeefe5b, numTrees=20, numFeatures=7,
 DecisionTreeRegressionModel: uid=DecisionTreeRegressor_b7bcb3bea997, depth=5, numNodes=63, numFeatures=7]

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


def get_features_vector(df): 
    return VectorAssembler(inputCols=df.columns[:-1], outputCol="features")

lr = LinearRegression(featuresCol='features', labelCol='ctr', maxIter=40, regParam=0.4, elasticNetParam=0.8)    
gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)    

features = get_features_vector(train_df)
pipeline = Pipeline(stages=[features])

gbt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: [features, gbt]}) \
        .addGrid(gbt.maxDepth, [2, 3, 4, 5])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()
#        .addGrid(gbt.numTrees, [3, 6, 9, 12, 15, 18, 20, 22])\

rf_ParamGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: [features, rf]}) \
        .addGrid(rf.maxDepth, [2, 3, 4, 5])\
        .addGrid(rf.numTrees, [19, 20, 21])\
        .build()

dt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: [features, dt]}) \
        .addGrid(dt.maxDepth, [4, 5, 6])\
        .addGrid(dt.minInfoGain, [0.0, 0.1, 0.2])\
        .addGrid(dt.maxBins, [25, 28, 32])\
        .build()
#        .addGrid(dt.numTrees, [3, 6, 9, 12, 15, 18, 20, 22])\

evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")

grids = gbt_paramGrid + rf_ParamGrid + dt_paramGrid

tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=grids,
                           evaluator=evaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(train_df)
model.bestModel


PipelineModel_d0347b6050b8

In [13]:
model.bestModel[1]

TypeError: 'PipelineModel' object is not subscriptable

In [14]:
model.validationMetrics

[0.3063465144953103,
 0.29273644193042986,
 0.27911211407467856,
 0.27127313791864127,
 0.16660811072257356,
 0.16167661114366336,
 0.15587172802154162,
 0.1516819300242907,
 0.10692592835763047,
 0.10478574521871672,
 0.10384882889125785,
 0.10244866802209669,
 0.08268294715321207,
 0.08226302253742422,
 0.08195370230129641,
 0.08146593986270258,
 0.3073562211904852,
 0.3431017878022488,
 0.33645473747267307,
 0.1945301742961752,
 0.216351416163912,
 0.1912762346948329,
 0.14782394302111623,
 0.15932760654806244,
 0.14927588303804062,
 0.12716719019940884,
 0.12782270695705375,
 0.12366418979531946,
 0.11586878324132778,
 0.11288248812602533,
 0.1107071222253666,
 0.26622204876468225,
 0.266360439578716,
 0.2715759195282558,
 0.34341053257282733,
 0.3431930942506459,
 0.3435626840916352,
 0.09688867685916056,
 0.09109996286664937,
 0.08577896721730169,
 0.26622204876468225,
 0.266360439578716,
 0.2715759195282558,
 0.34341053257282733,
 0.3431930942506459,
 0.3435626840916352,
 0.0964

In [17]:
model.getEstimatorParamMaps()


[{Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672,
   GBTRegressor_367dc82f24e4],
  Param(parent='GBTRegressor_367dc82f24e4', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
  Param(parent='GBTRegressor_367dc82f24e4', name='maxIter', doc='max number of iterations (>= 0).'): 4},
 {Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672,
   GBTRegressor_367dc82f24e4],
  Param(parent='GBTRegressor_367dc82f24e4', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
  Param(parent='GBTRegressor_367dc82f24e4', name='maxIter', doc='max number of iterations (>= 0).'): 5},
 {Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672

In [21]:
model.bestModel.stages[1]

GBTRegressionModel: uid=GBTRegressor_367dc82f24e4, numTrees=7, numFeatures=7

In [25]:
import numpy as np
model.getEstimatorParamMaps()[ np.argmin(model.validationMetrics) ]


{Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672,
  GBTRegressor_367dc82f24e4],
 Param(parent='GBTRegressor_367dc82f24e4', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='GBTRegressor_367dc82f24e4', name='maxIter', doc='max number of iterations (>= 0).'): 7}

In [27]:
np.min(model.validationMetrics)

0.08146593986270258

In [ ]:
val cvModel = cv.fit(training)
cvModel.getEstimatorParamMaps.zip(cvModel.avgMetrics)

In [32]:
df = model.bestModel.transform(test_df)

In [35]:
test_rmse = evaluator.evaluate(df)

In [36]:
print("Test data RMSE: %f" % round(test_rmse, 4))

Test data RMSE: 0.080700


In [43]:

from pyspark.ml.pipeline import PipelineModel

p_model = model
p_model.write().overwrite().save('spark_ml_model')
model = PipelineModel.load('spark_ml_model')
prediction = p_model.transform(test_df)


In [44]:
type(prediction)

pyspark.sql.dataframe.DataFrame

In [45]:
test_rmse = evaluator.evaluate(prediction)

In [46]:
print("Test data RMSE: %f" % round(test_rmse, 4))

Test data RMSE: 0.080700


In [53]:


get_paramGrids()

[('gbt',
  [{Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672,
     GBTRegressor_367dc82f24e4],
    Param(parent='GBTRegressor_367dc82f24e4', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
    Param(parent='GBTRegressor_367dc82f24e4', name='maxIter', doc='max number of iterations (>= 0).'): 4},
   {Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [VectorAssembler_f43983a5e672,
     GBTRegressor_367dc82f24e4],
    Param(parent='GBTRegressor_367dc82f24e4', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
    Param(parent='GBTRegressor_367dc82f24e4', name='maxIter', doc='max number of iterations (>= 0).'): 5},
   {Param(parent='Pipeline_bb80c029aa8b', name='stages', doc='a list of pipeline stages'): [V

In [54]:
type(tvs)

pyspark.ml.tuning.TrainValidationSplit

In [55]:
model1 = tvs.fit(train_df)


In [56]:
type(model1)

pyspark.ml.tuning.TrainValidationSplitModel

In [58]:
model1.bestModel

PipelineModel_7a9239c36c67

In [59]:
lr = LinearRegression(featuresCol='features', labelCol='ctr', maxIter=40, regParam=0.4, elasticNetParam=0.8)    

In [61]:
type(lr.params)

list

In [66]:
gbt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline.stages.}) \
        .addGrid(gbt.maxDepth, [2, 3, 4, 5])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()

TypeError: unsupported operand type(s) for +: 'Param' and 'GBTRegressor'

In [73]:
def dd(pipeline):
    pipeline_stages = pipeline.getStages()
    gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    gbt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [gbt]}) \
        .addGrid(gbt.maxDepth, [2, 3, 4, 5])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()

    
dd(pipeline)

In [70]:
pipeline.getStages()

[VectorAssembler_f43983a5e672]

In [81]:
import io
import sys
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.regression import LinearRegression, GBTRegressor, RandomForestRegressor, DecisionTreeRegressor

# Используйте как путь куда сохранить модель
MODEL_PATH = 'spark_ml_model'


def get_paramGrids_dict(pipeline):
    pipeline_stages = pipeline.getStages()
    
    lr = LinearRegression(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)    
    
    lr_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [lr]}) \
        .addGrid(lr.regParam, [0.4])\
        .addGrid(lr.maxIter, [40])\
        .addGrid(lr.elasticNetParam, [0.8])\
        .build()
    gbt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [gbt]}) \
        .addGrid(gbt.maxDepth, [2, 3, 4, 5])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()
    rf_ParamGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [rf]}) \
        .addGrid(rf.maxDepth, [2, 3, 4, 5])\
        .addGrid(rf.numTrees, [19, 20, 21])\
        .addGrid(rf.featureSubsetStrategy, ['onethird', '0.5', 'sqrt'])\
        .build()
    dt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [dt]}) \
        .addGrid(dt.maxDepth, [4, 5, 6])\
        .addGrid(dt.minInfoGain, [0.0, 0.1, 0.2])\
        .addGrid(dt.maxBins, [25, 28, 32])\
        .build()
    
    return {'lr': lr_paramGrid, 'gbt': gbt_paramGrid, 'rf': rf_ParamGrid, 'dt': dt_paramGrid }


def process(spark, train_data, test_data):
    #train_data - путь к файлу с данными для обучения модели
    train_df =  spark.read.parquet("train.parquet")
    #test_data - путь к файлу с данными для оценки качества модели
    test_df =  spark.read.parquet("test.parquet")

    features = VectorAssembler(inputCols=train_df.columns[:-1], outputCol="features")
    pipeline = Pipeline(stages=[features]) 
    # define models and paramGrids 
    paramGrids_dict = get_paramGrids_dict(pipeline)
    grids = paramGrids_dict['lr'] + paramGrids_dict['gbt'] + paramGrids_dict['rf'] + paramGrids_dict['dt']
    evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")

    # define tvs to find the best set of hyperparameters and best model
    tvs = TrainValidationSplit(estimator=pipeline,
                               estimatorParamMaps=grids,
                               evaluator=evaluator,
                               trainRatio=0.8)
    # train model and run tvs
    model = tvs.fit(train_df)
    p_model = model.bestModel
    print("Model and parameters: %s" % p_model.stages[1])
    print("Train data RMSE: %f" % round(np.min(model.validationMetrics), 4))
    
    # save model 
    p_model.write().overwrite().save('spark_ml_model')
    
    # evaluate model on test data 
    prediction = p_model.transform(test_df)
    test_rmse = evaluator.evaluate(prediction)
    print("Test data RMSE: %f" % round(test_rmse, 4))

def main(argv):
    train_data = argv[0]
    print("Input path to train data: " + train_data)
    test_data = argv[1]
    print("Input path to test data: " + test_data)
    spark = _spark_session()
    process(spark, train_data, test_data)


def _spark_session():
    return SparkSession.builder.appName('PySparkMLFitJob').getOrCreate()


if __name__ == "__main__":
    arg = sys.argv[1:]
    if len(arg) != 2:
        sys.exit("Train and test data are require.")
    else:
        main(arg)


Input path to train data: -f
Input path to test data: C:\Users\Al\AppData\Roaming\jupyter\runtime\kernel-97a87894-c6bd-475a-87b7-6f2611abf8b0.json
Model and parameters: GBTRegressionModel: uid=GBTRegressor_b41a347d3aac, numTrees=7, numFeatures=7
Train data RMSE: 0.081500
Test data RMSE: 0.080700


In [75]:
model

[Param(parent='GBTRegressor_367dc82f24e4', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'),
 Param(parent='GBTRegressor_367dc82f24e4', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'),
 Param(parent='GBTRegressor_367dc82f24e4', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'a

In [80]:
model1.params

[Param(parent='TrainValidationSplitModel_979a7bf272e6', name='estimator', doc='estimator to be cross-validated'),
 Param(parent='TrainValidationSplitModel_979a7bf272e6', name='estimatorParamMaps', doc='estimator param maps'),
 Param(parent='TrainValidationSplitModel_979a7bf272e6', name='evaluator', doc='evaluator used to select hyper-parameters that maximize the validator metric'),
 Param(parent='TrainValidationSplitModel_979a7bf272e6', name='seed', doc='random seed.'),
 Param(parent='TrainValidationSplitModel_979a7bf272e6', name='trainRatio', doc='Param for ratio between train and     validation data. Must be between 0 and 1.')]

In [82]:
def get_paramGrids_dict(pipeline):
    pipeline_stages = pipeline.getStages()
    
    lr = LinearRegression(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    gbt = GBTRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    rf = RandomForestRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction')
    dt = DecisionTreeRegressor(featuresCol='features', labelCol='ctr', predictionCol='prediction', maxDepth=5)    
    
    lr_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [lr]}) \
        .addGrid(lr.regParam, [0.4])\
        .addGrid(lr.maxIter, [40])\
        .addGrid(lr.elasticNetParam, [0.8])\
        .build()
    gbt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [gbt]}) \
        .addGrid(gbt.maxDepth, [5, 6, 7])\
        .addGrid(gbt.maxBins, [25, 28, 32])\
        .addGrid(gbt.maxIter, [4, 5, 6, 7])\
        .build()
    rf_ParamGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [rf]}) \
        .addGrid(rf.maxDepth, [2, 3, 4, 5])\
        .addGrid(rf.numTrees, [19, 20, 21])\
        .addGrid(rf.featureSubsetStrategy, ['onethird', '0.5', 'sqrt'])\
        .build()
    dt_paramGrid = ParamGridBuilder()\
        .baseOn({pipeline.stages: pipeline_stages + [dt]}) \
        .addGrid(dt.maxDepth, [4, 5, 6])\
        .addGrid(dt.minInfoGain, [0.0, 0.1, 0.2])\
        .addGrid(dt.maxBins, [32, 33])\
        .build()
    
    return {'lr': lr_paramGrid, 'gbt': gbt_paramGrid, 'rf': rf_ParamGrid, 'dt': dt_paramGrid }

#train_data - путь к файлу с данными для обучения модели
train_df =  spark.read.parquet("train.parquet")
    #test_data - путь к файлу с данными для оценки качества модели
test_df =  spark.read.parquet("test.parquet")

features = VectorAssembler(inputCols=train_df.columns[:-1], outputCol="features")
pipeline = Pipeline(stages=[features]) 
    # define models and paramGrids 
paramGrids_dict = get_paramGrids_dict(pipeline)
grids =  paramGrids_dict['gbt']  
    
evaluator = RegressionEvaluator(labelCol="ctr", predictionCol="prediction", metricName="rmse")
    # define tvs to find the best set of hyperparameters and best model
tvs = TrainValidationSplit(estimator=pipeline,
                               estimatorParamMaps=grids,
                               evaluator=evaluator,
                               trainRatio=0.8)
    # train model and run tvs
model = tvs.fit(train_df)
p_model = model.bestModel
print("Model: %s" % p_model.stages[1])
print("Train data RMSE: %f" % round(np.min(model.validationMetrics), 4))
    


Model: GBTRegressionModel: uid=GBTRegressor_236fe008517c, numTrees=6, numFeatures=7
Train data RMSE: 0.079700


In [106]:
model.bestModel.stages[1].extractParamMap() 

{Param(parent='GBTRegressor_236fe008517c', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='GBTRegressor_236fe008517c', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='GBTRegressor_236fe008517c', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regr